# Načítání a ukládání dat

V rámci hackathonu je možné, že budete muset načíst data z nějakých jiných zdrojů. Abyste pro to měli nějakou předlohu, tady načtu přiložený dataset a ukážu jak ho uložit a zpětně načíst do/z různých zdrojů.

Na načítání použiju `pandas`, protože je na to kvůli historii svého vývoje nejvhodnější.



In [24]:
import pandas

In [25]:
df = pandas.read_csv('scitani.csv')
# (write_csv taky existuje)

## json

In [26]:
df.to_json('scitani.json', orient='records')

In [27]:
pandas.read_json('scitani.json')

,ID,hodnota,stapro_kod,stapro_txt,datum,rok,obec_id
0,808433980,1148,2406,Počet obyvatel s trvalým nebo dlouhodobým pobytem,2011-03-26,2011,594881
1,808433981,659,2406,Počet obyvatel s trvalým nebo dlouhodobým pobytem,2011-03-26,2011,594890
2,808433982,621,2406,Počet obyvatel s trvalým nebo dlouhodobým pobytem,2011-03-26,2011,594911
3,808433983,1340,2406,Počet obyvatel s trvalým nebo dlouhodobým pobytem,2011-03-26,2011,594920
4,808433985,533,2406,Počet obyvatel s trvalým nebo dlouhodobým pobytem,2011-03-26,2011,594946
...,...,...,...,...,...,...,...
175076,742374420,106,2409,Počet domů,1890-12-31,1890,549746
175077,742379026,82,2409,Počet domů,1890-12-31,1890,552267
175078,742379040,55,2409,Počet domů,1890-12-31,1890,552194
175079,742379054,61,2409,Počet domů,1890-12-31,1890,547018


## SQL Databáze

Výhoda databáze je mimo jiné to, že pojme více tabulek, a s nimi umí dál pracovat, třeba rychleji než knihovny pythonu.

In [28]:
import sqlite3

In [39]:
df.to_sql('scitani', sqlite3.connect('scitani.db'), if_exists='replace')
pandas.read_csv('ciselnik.csv').to_sql('ciselnik', sqlite3.connect('scitani.db'), if_exists='replace')

6258

In [31]:
pandas.read_sql('SELECT * FROM scitani', sqlite3.connect('scitani.db'))

,index,ID,hodnota,stapro_kod,stapro_txt,datum,rok,obec_id
0,0,808433980,1148,2406,Počet obyvatel s trvalým nebo dlouhodobým pobytem,2011-03-26,2011,594881
1,1,808433981,659,2406,Počet obyvatel s trvalým nebo dlouhodobým pobytem,2011-03-26,2011,594890
2,2,808433982,621,2406,Počet obyvatel s trvalým nebo dlouhodobým pobytem,2011-03-26,2011,594911
3,3,808433983,1340,2406,Počet obyvatel s trvalým nebo dlouhodobým pobytem,2011-03-26,2011,594920
4,4,808433985,533,2406,Počet obyvatel s trvalým nebo dlouhodobým pobytem,2011-03-26,2011,594946
...,...,...,...,...,...,...,...,...
175076,175076,742374420,106,2409,Počet domů,1890-12-31,1890,549746
175077,175077,742379026,82,2409,Počet domů,1890-12-31,1890,552267
175078,175078,742379040,55,2409,Počet domů,1890-12-31,1890,552194
175079,175079,742379054,61,2409,Počet domů,1890-12-31,1890,547018


A ještě jeden pokročilejší dotaz, .

In [48]:
sqlite3.connect('scitani.db').execute("""
    SELECT SUM(hodnota), scitani.rok
    FROM scitani
    JOIN ciselnik
    ON scitani.obec_id = ciselnik.obec_id
    WHERE stapro_kod = 2406
    GROUP BY scitani.rok
    ORDER BY scitani.rok DESC;
""").fetchall()

[(10488891, 2011),
 (10230016, 2001),
 (10302164, 1991),
 (10291740, 1980),
 (9807596, 1970),
 (9571352, 1961),
 (8895314, 1950),
 (10672792, 1930),
 (10007864, 1921),
 (10074938, 1910),
 (9372091, 1900),
 (8664110, 1890),
 (8220815, 1880),
 (7563132, 1869)]

## HTTP request

Pro tento příklad musíte mít spuštěný fastapi backend v této složce, viz. `README.md`.

In [33]:
import requests

In [36]:
datas_link = "localhost:8000/data"

In [38]:
requests.get(datas_link).content

b'<html lang=en><meta charset=utf-8><meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width"><title>Error 404 (Not Found)!!1</title><style nonce="OCaKKNhZcq_og6dzPs19pA">*{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{color:#222;text-align:unset;margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px;}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}pre{white-space:pre-wrap;}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png)

In [37]:
pandas.read_html(datas_link)

HTTPError: HTTP Error 404: Not Found